In [ ]:
# ! git clone https://github.com/libindic/indic-trans.git
# ! cd indictrans && pip install -r requirements.txt && pip install .

In [1]:
from Lemmatization.lemmatizer import document_linearization
from json import load
import numpy as np
import time
import sys
from helper import read_train_data, load_train_data, load_concept_data, read_concept_data
import lsa
import tfidf
import esa
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khanm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
e:\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-18:11:35:35,294 INFO     [iwn.py:43] Loading bengali language synsets...
2023-12-18:11:39:40,633 INFO     [iwn.py:43] Loading bengali language synsets...
2023-12-18:11:39:48,310 INFO     [iwn.py:43] Loading bengali language synsets...


In [ ]:
trainDatasetPath = "../../Datasets/Anwesha/Train_Dataset/"
testDatasetPath = "../../Datasets/Anwesha/Test_Collection/"
conceptDatasetPath = "../../Datasets/Anwesha/New_Dataset_Concepts/"
docs, file_paths, dids = read_train_data(testDatasetPath, save_path=True)
c_docs, c_file_paths, c_dids = read_concept_data(conceptDatasetPath, save_path=True)

In [ ]:
TFIDF_vectorizer, tfidf_matrix = tfidf.set_up_TFIDF(docs, lemmatiser=False, save_model=True)
c_vectorizer, c_tfidf_matrix, doc_concept_matrix = esa.set_up_ESA(c_docs, TFIDF_vectorizer, tfidf_matrix, lemmatiser=False, save_model=True)
LSA_vectorizer, LSA_svd_transformer, svd_model, LSA_dvecs = lsa.set_up_LSA(docs, rank=600, lemmatiser=False, save_model=True)

TF-IDF model is saved in SavedModels directory.
ESA model is saved in SavedModels directory.


In [8]:
import pandas as pd

def read_excel_sheets(xls_path):
    sheet_names = ['Complexity_Level_1', 'Complexity_Level_2', 'Complexity_Level_3', 'Complexity_Level_4']
    df_queries = pd.DataFrame()
    for sheet in sheet_names:
        df = pd.read_excel(xls_path, sheet_name=sheet, header=0)
        df['Sno.'] = 'Q' + df['Sno.'].astype(str) + '_' + sheet[-1]
        df['complexity_level'] = int(sheet[-1])
        df.rename(columns={'Sno.': 'query_id', 'Query': 'query'}, inplace=True)
        df_queries = pd.concat([df_queries, df])
    return df_queries
df_queries = read_excel_sheets('../../Datasets/Anwesha/Queries.xlsx')
df_queries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 0 to 22
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   query_id          94 non-null     object
 1   query             94 non-null     object
 2   complexity_level  94 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ KB


In [9]:
df_queries.head()

,query_id,query,complexity_level
0,Q1_1,কালো কুমড়ো টাটকা দেখিয়ে দেব লবডঙ্কা,1
1,Q2_1,সমাজ হারানো লক্ষ্মীছাড়া,1
2,Q3_1,সে- শ্রীযুক্ত চারুচন্দ্র ভট্টাচার্য্য,1
3,Q4_1,তোকে দোষ দেয়া যায়না,1
4,Q5_1,তাহার বয়স যখন দুই বৎসর তখন তাহার কঠিন পীড়া হইল,1


In [11]:
from ranx import Qrels, Run, evaluate

run_dict = {}
for row in df_queries.itertuples():
    q_id = row.query_id
    q = row.query
    A, _ = tfidf.tfidf(TFIDF_vectorizer, tfidf_matrix, q, docs)
    k = 10
    x = np.argsort(A[0]).T[::-1][:k]
    index = 0
    q_dict = {}
    for y in x:
        if index < k:
            if A[0][y] > 0:
                q_dict['D'+str(dids[y][1])] = A[0][y]
        index = index + 1
    run_dict[q_id] = q_dict
run = Run(run_dict, name="anwesha-test")
qrel_df = pd.read_csv('../../Datasets/Anwesha/qrels.csv')
qrel_df['relevance'] = qrel_df['relevance'].astype('int64')
qrel = Qrels.from_df(
    df=qrel_df,
    q_id_col="query_id",
    doc_id_col="doc_id",
    score_col="relevance",
)

কালো কুমড়ো টাটকা দেখিয়ে দেব লবডঙ্কা
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.10164614 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.11684365 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.16092093 0.         0.         0.14317849 0.
  0.         0.20276824 0.         0.         0.         0.19512117
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.19307796 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.33333333 0.        

In [14]:
evaluate(qrel, run, ["map@10", "mrr@10", "ndcg@10"])

[('d:\\Data\\repos\\Bangla-IR-eval\\Models\\Anwesha\\../../Datasets/Anwesha/Train_Dataset/kolkata\\12001.txt',
  "মন্ত্রিত্ব দিলে দল ছাড়ার কথা বলেছিলেন কেন? ফেস অফে ভাঙলেন অনুব্রত নিজস্ব প্রতিবেদনঃ মন্ত্রী, বিধায়ক বা সাংসদ হতে চান না অনুব্রত মণ্ডল। কিন্তু কেন? জি ২৪ ঘণ্টার ফেস অফ অনুষ্ঠানেএসে সেই রহস্যের উপর থেকে পর্দা তুললেন খোদ তৃণমূলের বীরভূমের জেলা সভাপতিই। বললেন, সংগঠনের কাজেই খুশি তিনি। ফেস অফ অনুষ্ঠানেজি ২৪ ঘণ্টার এডিটর অনির্বাণ চৌধুরী অনুব্রতকে প্রশ্ন করেন, আপনি মন্ত্রী, বিধায়ক বা সাংসদ হতে চান না কেন? মন্ত্রিত্ব দিলেবলেন, দল ছেড়ে দেব? কীসের ভয়? নাকি দায়িত্ব এড়িয়ে যেতে চান? অনুব্রতর জবাব, একটা বিধায়ক, সাংসদদের থেকেও আমার দায়িত্ব বেশি।সাংসদ বা মন্ত্রী হলে বেশি মানুষকে আমি তার চারগুণ বেশি পাচ্ছি। সাংসদ, বিধায়ক হলে গ্যারেজ হয়ে যাব। আমায় দিদি কোনওদিনইফেরাননি। অনেক মানুষের সেবা করেছি। আপনাদের আশীর্বাদ থাকলে আরও পারব। পশ্চিমবঙ্গে তৃণমূল ৪২-এ ৪২ পাবে বলেও দাবি করলেন অনুব্রত মণ্ডল।কিন্তু লোকসভা ভোটে ২২টি আসনে জেতার টার্গেট করেছে বিজেপি। আপনার জেলার আসনও রয়েছে? তৃণমূলের জেলা সভাপতির হুঁ